Load Environment

In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
import random
from pandas import read_csv, DataFrame
import numpy as np
import matplotlib.pyplot as plt


Load Data